# Prompt Walkthrough

An overview of the different types of prompts in LangChain and how to use them

### Basic PromptTemplate

The most simple type of prompt - a string template that takes any number of input variables. The template should be formatted as a Python f-string.

In [1]:
from langchain.prompts import PromptTemplate

In [2]:
# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
no_input_prompt.format()

'Tell me a joke.'

In [4]:
# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["adjective"], template="Tell me a {adjective} joke.")
one_input_prompt.format(adjective="funny")

'Tell me a funny joke.'

In [7]:
# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(input_variables=["adjective", "content"], template="Tell me a {adjective} joke about {content}.")
multiple_input_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

### Examples
Examples are datapoints that can be used to show the model how to produce results. They can be either strings, or dictionaries that are then turned into strings by an example prompt itself.

In [12]:
examples = [{"input": "happy", "output": "sad"}, {"input": "tall", "output": "short"}]
example_prompt = PromptTemplate(input_variables=["input","output"], template="Input: {input}\nOutput: {output}")

### Few Shot prompts

We can then use these examples to construct few shot prompts.

In [13]:
from langchain.prompts.few_shot import FewShotPromptTemplate

In [14]:
prompt_from_string_examples = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
)
print(prompt_from_string_examples.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


### LengthBased ExampleSelector

We also do more clever things with prompts - for example, only select a certain number of examples in order to limit the size of the text passed in. This will vary with the input text size.

In [15]:
from langchain.prompts.example_selector.length_based import LengthBasedExampleSelector

In [31]:
example_selector = LengthBasedExampleSelector(
    examples=examples, example_prompt=example_prompt, max_length=8
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
)

In [32]:
# An example with small input, so it selects both examples.
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: happy
Output: sad

Input: big
Output:


In [33]:
# An example with long input, so it selects only one example.
print(dynamic_prompt.format(adjective="big and huge and massive"))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive
Output:


# Similar Prompt

Besides selecting a variable number of examples to show, we can also select examples that most closely match the user input. This is done by creating embeddings of the user input and comparing it embeddings of the examples.

In [34]:
from langchain.prompts.example_selector.semantic_similarity import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [35]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, OpenAIEmbeddings(), FAISS, k=1
)
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
)

In [37]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [38]:
# Input is a measurment, so should select the tall/short example
print(similar_prompt.format(adjective="fat"))

Give the antonym of every input

Input: tall
Output: short

Input: fat
Output:
